# Fortgeschrittene Empfehlungssysteme

Willkommen zum Code Notebook zum Erstellen von fotgeschrittenen Empfehlungssystemen mit Python. Das ist ein optionales Notebook für euch. Aktuell gibt es hierzu keine Video-Lektion. Das liegt daran, dass wir ein hohes Niveau an statistischen/mathematischen Methoden und viel SciPy verwenden.

Empfehlungssyteme benötigen üblicherweise größere Datensätze und eine spezielle Struktur. Deshalb haben wir zu diesem Thema kein eigenständiges Projekt. Stattdessen haben wir für euch diesen intensiven Walktrough geschrieben. Erneut werden wir mit Python ein Empfehlugnssystem auf den Filmdaten aufbauen.

*Hinweis: Die tatsächliche Mathematik hinter Empfhelungssystemen ist ziemlich komplizierte lineare Algebra.*

___

## Methoden

Die beiden am häufigsten verwendeten Methoden für Empfehlugnssysteme sind **Content-Based** und **Collaborative Filtering (CF)**.

* Collaborative Filtering erstellt Emfehlungen basierend auf dem Wissen, welche Einstellung ein Nutzer gegenüber Objekten hat. Es nutzt also die "Weisheit der Masse" um passende Objekte zu empfehlen.

* Content-based Empfehlungssysteme konzentrieren sich auf die Eigenschaften von Objekten und erstellen die Empfehlungen anhand derer Ähnlichkeit.

## Collaborative Filtering

Im Allgemeinen werden CF Systeme häufiger verwendet als Content-based Systeme.  Das liegt daran, dass sie üblicherweise für den Nutzer relevantere Ergebnisse erzeugen. Außerdem ist es relativ einfach zu verstehen (betrachtet man die Implementierung im Allgemeinen). Der Algorithmus hat die Fähigkeit selbstständig "Feature-Learning" zu betreiben und so mehr darüber zu lernen, welche Eigenschaften er verwenden soll.

CF kann darüberhinaus in **Memory-based CF** und **Model-based CF** unterteilt werden.

In diesem Tutorial werden wir ein Model-based CF System implimentieren indem wir Single Value Decomposition (SVD) verwenden. Das Momory-based CF System bauen wir durch die Berechnung von Kosinus-Ähnlichkeiten auf.

## Die Daten

Wir werden den berühmten MovieLens Datensatz verwenden, der einer de am häufigsten verwendeten Datansaätze zum Implementieren und Testen von EMpfehlungssystemen ist. Er enthält 100k Filmbewertungen von 943 Nutzern für eine Auswahl an insgesamt 1682 Filmen. 

Du kannst einfach die u.data Datei verwenden, die im Lektionsverzeichnis hinterlegt ist.

___

## Der Start

Importieren wir zunächst einige Libraries, die wir brauchen werden:

In [1]:
import numpy as np
import pandas as pd

Wir können als nächstes die u.data Datei lesen, die den vollständigen Datensatz enthält. Eine kurze Beschreibung des Datensatzes findest du [hier](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

Achte darauf, dass wir Tab als Trennzeichen definieren.

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

Schauen wir uns die Daten kurz an:

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


Siehst du, dass wir nur die item_id vorliegen haben, nicht den Filmtitel. Wir können die Movie_ID_Titles CSV verwenden, um die Filmnamen auszulesen und sie mit unserem DataFrame zu mergen.

In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


Nun das Mergen:

In [5]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


Jetzt schauen wir uns einmal an, wie viele unique Nutzer und Filme es gibt.

In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Anzahl an Nutzern: '+ str(n_users))
print('Anzahl an Filmen: '+str(n_items))

Anzahl an Nutzern: 944
Anzahl an Filmen: 1682


## Train Test Split

Empfehlungssysteme sind aufgrund ihrer Natur sehr schwer auszuwerten. Wir werden trotzdem einen Split für die Daten vornehmen, um sie im Rahmen dieses Tutorials bewerten zu können. Um dies zu tun werden wir den Datensatz in zwei Sets aufteilen. Allerdings verwenden wir nicht unseren üblichen X_train,X_test,y_train,y_test Split. Wir können die Daten einfach in zwei Sets aufteilen:

In [7]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

/Users/davidmika/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering Ansätze können in zwei Hauptsektionen geteilt werden: **user-item filtering** und **item-item filtering**.

Ein *user-item filtering* nimmt einen bestimmten Nutzer und findet Nutzer, die ihm möglichst ähnlich in ihren Bewertungen sind. Dann empfihelt er Objekte, die diese ähnlichen Nutzer mochten.

Ein *item-item filtering* nimmt ein Objekt und findet Nutzer, die dieses Objekt mochten. Dann werden Objekte herausgesucht, die diese Nutzer auch mochten. 

* *Item-Item Collaborative Filtering:* "Nutzer, die das mochten, mochten auch..."
* *User-Item Collaborative Filtering:* "Nutzer, die dir ähnlich sind, mochten..."

In beiden Fällen wird eine User-Item-Matrix erstellt, die auf dem Gesamtdatensatz aufbaut. 

Da wir den Datensatz in Trainings- und Testset aufgeteilt haben müssen wir auch zwei Matrizen im Format `943 x 1682` erstellen (alle Nutzer x alle Filme).

Die Trainings-Matrix enthält dan 75% der Daten und die Test-Matrix die restlichen 25%.

Beispielhafte User-Item-Matrix:
<img class="aligncenter size-thumbnail img-responsive" src="http://s33.postimg.org/ay0ty90fj/BLOG_CCA_8.png" alt="blog8"/>

Nachdem du die User-Item-Matrix aufgebaut hast kannst du die Ähnlichkeiten berechnen und eine Ähnlichkeits-Matrix erstellen.

Die Ähnlichkeitswerte zwischen den Objekten beim Item-Item CF werden berechnet, indem alle Nutzer betrachtet werden, die beide Objekte bewertet haben.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/i522ma83z/BLOG_CCA_10.png"/>

Für User-Item CF werden die Ähnlichkeitswerte zwischen den Nutzern berechnet, indem alle Objekte betrachtet werden, die beide Nutzer gesehen haben.

Eine Distanzmetrik die häufig in Empfehlungssystemen verwendet wird ist die *Kosinus-Ähnlichkeit*. Dabei werden die Bewertungen als Vektoren im n-dimensionalen Raum gesehen und die Ähnlichekit durch den Winkel zwischen ihnen berechnet. Kosinus-Ähnlichkeit für Nutzer a und m wird durch die nachfolgende Formel berechnet werden. 

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

Um die Ähnlichekti zwischen den Objekten m und b zu berechnen dient folgende Formel:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Dein erster Schritt wird es sein, die User-Item-Matrix zu erstellen. Da wir sowohl Test- als auch Trainingsdaten haben, müssen wir zwei Matrizen erstellen.

In [8]:
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

Wir können die [`pairwise_distance`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) Funktion aus SciKit Learn verwenden, um die Kosinus-Ähnlichkeit zu berechnen. Achte daruf, dass das Ergebnis zwischen 0 und 1 liegen wird, da alle Bewertungen positiv sind.

In [9]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Als nächstes werden wir Vorhersagen treffen. Wir haben bereits die Ähnlichkeits-Matrizen erstellt: `user_similarity` und `item_similarity`. Damit können wir eine Vorhersage treffen indem wir für User-based CF folgende Formel verwenden:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

Dabei schauen wir auf die Ähnlichkeit zwischen Nutzer k und a als Gewichte die mit den Ratings eines gleichen Nutzers a multipliziert werden (korrigiert um die durchschnittliche Bewertung des Nutzers). Dies muss zusätzlich normalisiert werden, sodass die Bewertungen zwischen 1 und 5 bleiben. Zu guter Letzt summiert man die durchschnittlichen Bewertungen des Nutzers, den man vorhersagen möchte.

Die Idee dahinter ist, dass manche Nutzer dazu tendieren könnten, immer hohe oder immer niedrige Bewertungen zu geben. Der relative Unterschied zwischen den Bewertungen, die diese Nutzer abgeben, ist wichtiger als der absolute Unterschied. Um das zu veranschaulichen: ein Nutzer k gibt seinem Lieblingsfilm 4 Sterne und allen anderen guten Filmen 3 Sterne. Nutzer t bewertet Filme die er mag mit 5 Sternen, wobei er selbst langweiligen Filmen immer mindestens 3 Sterne gibt. Beide könnten die gleichen Filme mögen, behandeln aber das Bewertungssystem unterschiedlich.

Wenn wir Item-based CF Vorhersagen treffen, dann müssen wir nicht um die durchschnittliche Nutzerberwertung korigieren. Jeder einzelne Nutzer selbst wird genutzt, um die Vorhersagen zu treffen.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [10]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #Wir nutzen np.newaxis damit mean_user_rating das selbe Format wie Ratings hat
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [11]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

## Auswertung

Es gibt viele Auswertungsmetriken. Eine der meist verwendeten für die Genauigkeit der vorhergesagten Bewertungen ist der *Root Mean Squared Error* (RMSE). 

<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

Wir können die [mean_suquare_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE) Funktion von `sklearn` verwenden. Dabei ist der RMSE einfach die Quadratwurzel von MSE.

Da wir nur die vorhergesagten Bewertungen betrachten wollen, die in den Testdaten sind, filtern wir alle anderen Werte aus der Vorhersage-Matrix mit `prediction[ground_truth.nonzero()]`.

In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [13]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1244140039203865
Item-based CF RMSE: 3.450851108963224


Memory-based CF Algorithmen sind leicht zu implementieren und ergeben vernünftige Vorhersagen. Der Nachteil an Memory-based CF ist, dass es nicht skalierbar auf Echt-Welt Anwendungen ist und nicht das bekannte "Kaltstart"-Problem löst (neue Nutzer oder Items im System). Model-based CF Methoden sind skalierbar und können mit viel weniger Daten als Memory-based Modelle arbeiten. Allerdings haben auch sie Probleme mit neuen Einträgen im System, die gar keine Bewertungen haben.

## Model-based Collaborative Filtering

Model-based CF basiert auf Matrix-Faktorisierung (MF), welche größere Aufmerksamkeit erhalten hat. Vor allem deshalb, da es als Unsupervised Learning Algorithmus für latente Variablen und Dimensionsreduktion verwendet wird. MF wird viel für Empfehlungssysteme genutzt, da es besser mit Skalierbarkeit und Seltenheit (en. sparsity) umgehen kann als Memory-based CF. Das Zeil von MF ist es, die lateten Präferenzen von Nutzern und latenten Atribute von Items durch die gegebenen Bewertungen zu lernen. So können dann unbekannte Bewertungen durch das Produkt der latenten Eigenschaften von Nutzern und Objekten berechnet werden. Wenn man eine sehr kanpp befüllte Matrix hat, die viele Dimensionen hat, können wir durch MF eine Restrukturierung durchführen. Und wir können die Matrix als Multiplikation zweier restrukturierten Matrizen mit niedrigerem Rang erhalten. Darin enthlatne die Zeilen den latenten Vektor. Wir passen diese Matrizen dann an, um unsere Original-Matrix möglichst genau wiederzuerhalten. Durch Multiplikation erhalten wir dann auch die fehlenden Werte der Original-Matrix.

Berechnen wir nun das Seltenheits-Niveau der MovieLens Daten:

In [15]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('Das Seltenheits-Niveau ist ' +  str(sparsity*100) + '%')

Das Seltenheits-Niveau ist 93.7%


Um ein Beispiel der gelernten latenten Präferenzen von Nutzern und Items zu geben: sagen wir der MovieLens Datensatz enthält die foglgende Information: (user id, age, location, gender, movie id, director, actor, language, year, rating). Durch die Anwedung von MF lernt das Modell, dass die wichtigen Eigenschaften die Altersgruppe (en. age group), Ort (en. location) und das Geschlecht (en. gender) sind. Und von den Filmeigenschaften sind es Dekade (en. decade), Direktor (en. director) und Schauspieler (en. actor) am wichtigsten sind. Wenn wir uns jetzt die Informationen anschauen, die wir im Datensatz haben, sehen wir, dass wir die Dekade nicht vorliegen haben. Das Modell jedoch kann es selbst lernen. Das wichtige ist, dass das CF Modell nur vorhanden Daten nutzt, um latente Eigenschaften zu lernen. Je mehr Daten vorhanden sind, desto besser kann das Modell lernen und performen.

Modelle die sowohl Bewertungen als auch Eigenschaften verwenden werden **Hybrid Recommender Sysmtes** genannt, wo sowohl CF als auch Content-based Modelle kombinert werden. Sie zeigen üblicherweise eine höhere Genauigkeit als die beiden anderen Varianten allein.

## SVD

Eine gut bekannte MF Methode ist **Singular Value Decomposition** (SVD). CF kann formuliert werden, indem eine Matrix `X` durch SVD geschätzt wird. Das siegreiche Team beim Netflix Prize Wettbewerb nutzte SVD MF Modelle, um Produktempfehlungen zu erstellen. Mehr Infos liefern diese beiden Artikel: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) und [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).

Die allgemeine Gleichung kann wiefolgt ausgedrückt werden:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />

Gegeben `m x n` Matrix `X`:
* *`U`* ist eine *`(m x r)`* orthogonale Matrix
* *`S`* ist eine *`(r x r)`* diagonale Matrix
* *V^T* ist eine *`(r x n)`* orthogonale Matrix

Elemente auf der Diagonalen `S` sind bekannt als *Singular Values von `X`*. 

Die Matrix *`X`* kann zu `U`*, *`S`* und *`V`* faktorisiert werden. Die *`U`* Matrix repräsentiert die Feature-Vektoren. die den Items in den versteckten Features entsprechen.
<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/kwgsb5g1b/BLOG_CCA_5.png"/>

Jetzt kann man eine Vorhersage durch das Produkt von *`U`*, *`S`* und *`V^T`* durchfürhen.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/ch9lcm6pb/BLOG_CCA_4.png"/>

In [16]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# Erhalte die SVD Komponenten von der Trainigns-Matrix. Wähle k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.710319447968387


Einfach die wenig bekannten Feautres zu addressieren ist sehr anfällig für Overfitting. SVD kann sehr langsam und rechnungsintensiv sein. Neuere Arbeit minimiert den squared Error durch die Anwendung von Least Squares oder stochastischen Gradienten-Abstieg.

# Gut gemacht!